# Atlas de l'expérience `keyexp`

In [ ]:
filename = 'resultat.nc'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import cartopy.crs as ccrs
from netCDF4 import Dataset

%matplotlib inline

data = Dataset(filename)

| Variable | Notation formelle usuelle | Nom dans `resultat.nc` | Unité | 
|:---------|:-----------------|:-----------------------|:------|
| Température de surface | $T_s(x,y,t)$ | `tsurf` | K |
| Pression de surface | $p_s(x,y,t)$ | `ps` | Pa |
| Pression atmosphérique | $p(x,y,z,t)$ | `p` | Pa |
| Température de l'atmosphère | $T(x,y,z,t)$ | `temp` | K |
| Température potentielle | $\theta(x,y,z,t)$ | `teta` | K |
| Vent zonal | $u(x,y,z,t)$ | `u` | m s$^{-1}$ |
| Vent méridien | $v(x,y,z,t)$ | `v` | m s$^{-1}$ |
| Vent vertical | $w(x,y,z,t)$ | `w` | m s$^{-1}$ |
| Albédo | $A_b(x,y,t)$ | `ALB` | SU |
| Flux solaire incident au sommet | $F_{vis}^{\downarrow_{TOA}}(x,y,t)$ | `ISR` | W m$^{-2}$ |
| Flux solaire absorbé par la surface | $F_{vis}^{\downarrow_{Surf}}(x,y,t)$ | `ASR` | W m$^{-2}$ |
| Flux infrarouge émis au sommet | $F_{IR}^{\uparrow_{TOA}}(x,y,t)$ | `OLR` | W m$^{-2}$ |
| Taux de chauffage SW | $Q_{rad}^{SW}(x,y,z,t)$ | `zdtsw` | K s$^{-1}$ |
| Taux de chauffage LW | $Q_{rad}^{LW}(x,y,z,t)$ | `zdtlw` | K s$^{-1}$ |
| Taux de chauffage total | $Q_{rad}(x,y,z,t)$ | `dtrad` | K s$^{-1}$ |
| RMM en vapeur d'eau | $q_{vap}(x,y,z,t)$ | `h2o_vap` | kg kg$^{-1}$ |
| RMM en glace d'eau | $q_{glace}(x,y,z,t)$ | `h2o_ice` | kg kg$^{-1}$ |
| Masse de glace au sol par m$^2$ | $m_{glace}(x,y,t)$ | `h2o_ice_surf` | kg m$^{-2}$ |

Quelques remarques : 
+ SW signifie shortwave, donc correspond au domaine des courtes longueurs d'onde (appelé aussi domaine "visible");
+ De la même façon LW signifie longwave, et correspond au domaine des grandes longueurs d'onde ou domaine infrarouge;
+ RMM signifie rapport de mélange **massique**, en kg de l'espèce considérée par kg d'air;
+ Le vent $\vec{v}$ s'écrit suivant les trois directions de l'espace $\vec{v}=u\ \hat{x}+v\ \hat{y}+w\ \hat{z}$.


In [ ]:
longitude=data.variables['longitude'][:]
latitude=data.variables['latitude'][:]
altitude=data.variables['altitude'][:]
Time = data.variables['Time'][:]
Ls = data.variables['Ls'][:]

$L_s$ est la longitude solaire, qui sert à désigner la position de la planète sur son orbite. Cumul des années pour les représentations graphiques :

In [ ]:
dafirst = Time[0]
daint = Time[1] - dafirst
dalast = dafirst + (len(Time)-1)*daint
year = 0.
add = np.linspace(dafirst,dalast,num=len(Time)) ; add[0] = 0.
for iii in range(1,len(Ls)):
    if Ls[iii] - Ls[iii-1] < 0: year = year+1.
    add[iii] = year*360.
Ls_true = add + Ls

## Topographie

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo=True # ajouter les traits de côte actuels
interpolation=False # voir les mailles réelles ou interpolées

# Code ------------------------------------------------------------

import matplotlib as mpl

grav=9.81 # acceleration gravitationnelle
phis=data.variables['phisinit'][:]
dataplt = phis/grav


fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
if earthtopo : ax.coastlines()
ax.set_global()
clevs = np.arange(0,4100,100)
gl = ax.gridlines(linestyle='--',color='black',draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
colormap = plt.cm.get_cmap('terrain')


if interpolation :
    plt.contourf(longitude, latitude, dataplt, clevs, transform=ccrs.PlateCarree(),cmap=colormap)
    cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)


else :
    fill = ax.pcolormesh(longitude, latitude, dataplt, transform=ccrs.PlateCarree(),cmap=colormap, shading = "nearest",norm=mpl.colors.BoundaryNorm(clevs, ncolors=colormap.N, clip=False))
    cb = fig.colorbar(fill, extend="max", pad=0.02, aspect=16, shrink=0.8)
    
    
cb.set_label(r'mètres',size=12,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.title('Topographie', size=14)    
plt.show()

## Cycle saisonnier d'une variable à différentes latitudes

In [ ]:
# Paramètres utilisateurs -----------------------------------------
varname = 'tsurf' # nom de la variable
vmin = 160
vmax = 340
# Code ------------------------------------------------------------

var = data.variables[varname][:,:,:]
fig, ax = plt.subplots(figsize=(12,8))

ilat1 = np.where(abs(latitude-0.)==
                  abs(latitude-0.).min())[0]
ilat2 = np.where(abs(latitude+25.)==
                  abs(latitude+25.).min())[0]
ilat3 = np.where(abs(latitude+50.)==
                  abs(latitude+50.).min())[0]
ilat4 = np.where(abs(latitude+75.)==
                  abs(latitude+75.).min())[0]

# Using set_dashes() to modify dashing of an existing line
ts1, = ax.plot(Ls_true, np.mean(var[:,ilat1,:],axis=2), label='Latitude '+str(latitude[ilat1]))
ts2, = ax.plot(Ls_true, np.mean(var[:,ilat2,:],axis=2), label='Latitude '+str(latitude[ilat2]))
ts3, = ax.plot(Ls_true, np.mean(var[:,ilat3,:],axis=2), label='Latitude '+str(latitude[ilat3]))
ts4, = ax.plot(Ls_true, np.mean(var[:,ilat4,:],axis=2), label='Latitude '+str(latitude[ilat4]))
ax.set_ylim([vmin,vmax])
ax.set_xlabel('Solar longitude Ls')
ax.set_ylabel(varname)
ax.grid()
ax.legend()

## Evolution en moyenne globale

In [ ]:
# Paramètres utilisateurs -----------------------------------------
varname = 'tsurf' # nom de la variable
# Code ------------------------------------------------------------

var = data.variables[varname][:,:,:]

fig, ax = plt.subplots(figsize=(12,8))
area = data.variables['aire'][:]
totarea = np.sum(np.sum(area[:,:],axis=0),axis=0)
varmeanarea = np.mean(np.mean(var[:,:,:]*area,axis=1),axis=1)/totarea*area.size
ts1, = ax.plot(Ls_true, varmeanarea)
ax.set_xlabel('Solar longitude Ls')
ax.set_ylabel(varname)
ax.set_title("Evolution moyenne sur l'ensemble du globe")
ax.grid()

## Bilan énergétique

Quatre figures donnant en fonction du temps $L_s$ et de la latitude :
1. le flux solaire incident au sommet $F_{vis}^{\downarrow_{TOA}}$,
2. le flux solaire absorbé par la surface $F_{vis}^{\downarrow_{Surf}}$,
3. $F_{IR}^{\uparrow_{TOA}}$ le flux infrarouge émis au sommet,
4. la température de surface $T_s$.

In [ ]:
ISR=data.variables['ISR'][:,:,:]
ASR=data.variables['ASR'][:,:,:]
OLR=data.variables['OLR'][:,:,:]
tsurf=data.variables['tsurf'][:,:,:]
ncontour = 20

fig, axs = plt.subplots(2, 2,figsize=(12,10))
im1 = axs[0, 0].contourf(Ls_true,latitude,np.mean(ISR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
#axs[0, 0].cbar()
axs[0, 0].set_ylim([-90,90])
axs[0, 0].set_title('Incoming Solar Radiation')
fig.colorbar(im1,ax=axs[0, 0])

im2 = axs[0, 1].contourf(Ls_true,latitude,np.mean(ASR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
axs[0, 1].set_title('Absorbed Solar Radiation')
fig.colorbar(im2,ax=axs[0, 1])

im3 = axs[1, 0].contourf(Ls_true,latitude,np.mean(OLR[:,:,:],axis=2).transpose(),
                         ncontour,cmap='jet')
axs[1, 0].set_title('OLR')
fig.colorbar(im3,ax=axs[1, 0])


im4 = axs[1, 1].contourf(Ls_true,latitude,(np.mean(tsurf[:,:,:],axis=2)-273).transpose(),
                         ncontour,cmap='bwr',vmin= -90,vmax=60)
axs[1, 1].set_title('Tsurf')
fig.colorbar(im4,ax=axs[1, 1])

for ax in axs.flat:
    ax.set(xlabel='Ls', ylabel='Latitude')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

## Carte en moyenne temporelle sur la totalité de l'expérience

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo = False # ajouter les traits de côte actuels
varname = 'tsurf'
vmin = 180
vmax = 330

# Code ------------------------------------------------------------
dataplt = data.variables[varname][:,:,:]

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
if (earthtopo): ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(vmin,vmax,29)
plt.contourf(longitude, latitude, np.mean(dataplt[:,:,:],axis=0),
             clevs, transform=ccrs.PlateCarree(),cmap="jet")
plt.title(r"Température de surface moyenne", size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'K',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()

## Coupe zonale à une longitude solaire $L_s$ donnée

In [ ]:
# Paramètres utilisateurs -----------------------------------------
year_user = 1 # année de simulation à regarder
Ls_user = 90. # longitude solaire choisie

varname = 'temp'
vmin = 180.
vmax = 360.

# Code ------------------------------------------------------------
Ls_true_user = year_user*360. + Ls_user

Ls_ind = np.where(abs(Ls_true-Ls_true_user)==
                  abs(Ls_true-Ls_true_user).min())[0]
print("La valeur la plus proche trouvée est Ls = "
      + str(Ls_true[Ls_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

# Code ------------------------------------------------------------
var = data.variables[varname][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=latitude
yplt = np.squeeze(np.mean(np.mean(p[Ls_ind,:,:,:],axis=3),axis=2))
dataplt = np.squeeze(np.mean(var[Ls_ind,:,:,:],axis=3))
im = ax.contourf(xplt,yplt,dataplt,60,cmap='jet',vmax=vmax, vmin=vmin)
#axs[0, 0].cbar()
#ax1.set_ylim([-90,90])
ax.set_title(varname)
ax.set_yscale('log')
ax.set(xlabel='Latitude', ylabel='p')
ax.invert_yaxis()
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------

### Carte à une longitude solaire $L_s$ donnée

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo = False # ajouter les traits de côte actuels

year_user = 1 # année de simulation à regarder
Ls_user = 90. # longitude solaire choisie

varname = 'tsurf'
vmin = 180.
vmax = 360.

# Code ------------------------------------------------------------
Ls_true_user = year_user*360. + Ls_user

Ls_ind = np.where(abs(Ls_true-Ls_true_user)==
                  abs(Ls_true-Ls_true_user).min())[0]
print("La valeur la plus proche trouvée est Ls = "
      + str(Ls_true[Ls_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

# Code ------------------------------------------------------------

var = data.variables[varname][:,:,:]
dataplt = var

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
if (earthtopo): ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(vmin,vmax,29)
plt.contourf(longitude, latitude, np.squeeze(dataplt[Ls_ind,:,:]),
             clevs, transform=ccrs.PlateCarree(),cmap="jet")
plt.title(r"Température de surface", size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'K',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()

### Moyenne zonale d'un champ à une altitude donnée

In [ ]:
# Paramètres utilisateurs -----------------------------------------
alt_user = 1 # altitude choisie en km
vmin = 180. # bornes de l'echelle de couleur, a ajuster pour 
vmax = 360. # une meilleure visualisation
varname = 'temp'

# Code
# -----------------------------------------------------------------
altitude = data.variables['altitude'][:]
alt = data.variables['altitude'][:]
alt_ind = np.where(abs(alt-alt_user)==
                  abs(alt-alt_user).min())[0]
print("La valeur la plus proche trouvée est altitude = "
      + str(alt[alt_ind]))

var = data.variables[varname][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=Ls_true
yplt = latitude
dataplt = np.squeeze(np.mean(var[:,alt_ind,:,:],axis=3))
im = ax.contourf(xplt,yplt,dataplt.transpose(),60,cmap='jet',
                 vmax=vmax, vmin=vmin)
ax.set_title("Moyenne zonale de "+varname+" (alt="+str(alt[alt_ind])+")")
ax.set(xlabel='Solar longitude (Ls)', ylabel='Latitude')
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------

### Carte d'un champ à Ls et z donnés

In [ ]:
# Paramètres utilisateurs -----------------------------------------
alt_user = 1. # l'altitude a laquelle on veut regarder la variable 4D
year_user = 1 # année de simulation à regarder
Ls_user = 130.

varname = 'temp'
vmin = 180. # bornes de l'echelle de couleur, a ajuster pour 
vmax = 360. # une meilleure visualisation

# Code ------------------------------------------------------------
alt = data.variables['altitude'][:]
alt_ind = np.where(abs(alt-alt_user)==
                  abs(alt-alt_user).min())[0]
print("La valeur la plus proche trouvée est altitude = "
      + str(alt[alt_ind]))

Ls_true_user = year_user*360. + Ls_user

Ls_ind = np.where(abs(Ls_true-Ls_true_user)==
                  abs(Ls_true-Ls_true_user).min())[0]
print("La valeur la plus proche trouvée est Ls = "
      + str(Ls_true[Ls_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

var = data.variables[varname][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=longitude
yplt = latitude
dataplt = np.squeeze(np.mean(var[Ls_ind,alt_ind,:,:],axis=0))
im = ax.contourf(xplt,yplt,dataplt,60,cmap='jet',
                 vmax=vmax, vmin=vmin)
ax.set_title("Carte de "+varname)
ax.set(xlabel='Longitude', ylabel='Latitude')
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------

## Série temporelle en un point

In [ ]:
# Paramètres utilisateurs -----------------------------------------
longi_user = 160. # longitude considérée
lati_user = 90. # latitude considérée
alt_user = 1. # l'altitude a laquelle on veut regarder la variable 4D

varname = 'temp'
vmin = 180. # bornes de l'echelle de couleur, a ajuster pour 
vmax = 360. # une meilleure visualisation

# On regarde le champ de température 4D "temp"
# -----------------------------------------------------------------
alt = data.variables['altitude'][:]
alt_ind = np.where(abs(alt-alt_user)==
                  abs(alt-alt_user).min())[0]
print("La valeur la plus proche trouvée est altitude = "
      + str(alt[alt_ind]))
longi = data.variables['longitude'][:]
lon_ind = np.where(abs(longi-longi_user)==
                  abs(longi-longi_user).min())[0]
print("La valeur la plus proche trouvée est longitude = "
      + str(longi[lon_ind]))
lati = data.variables['latitude'][:]
lat_ind = np.where(abs(lati-lati_user)==
                  abs(lati-lati_user).min())[0]
print("La valeur la plus proche trouvée est latitude = "
      + str(lati[lat_ind]))
# -----------------------------------------------------------------
var = data.variables[varname][:,:,:,:]

fig, ax = plt.subplots(figsize=(12,8))
dataplt = var[:,alt_ind,lat_ind,lon_ind]
plt1, = ax.plot(Ls_true, dataplt)
ax.set_xlabel('Solar longitude Ls')
ax.set_ylabel('Température (K)')
ax.set_title("Evolution de "+varname)
ax.grid()

## Carte de la valeur moyenne d'un champ dans un intervalle de $L_s$

In [ ]:
# Paramètres utilisateurs -----------------------------------------
earthtopo = False # ajouter les traits de côte actuels
year_user = 1 # année de simulation à regarder
Ls_user_min = 120. # longitude solaire choisie
Ls_user_max = 140. # longitude solaire choisie

varname = 'tsurf'
vmin = 180
vmax = 330

# Code ------------------------------------------------------------
Ls_true_user_min = year_user*360. + Ls_user_min
Ls_true_user_max = year_user*360. + Ls_user_max

Lsmin_ind = np.where(abs(Ls_true-Ls_true_user_min)==
                  abs(Ls_true-Ls_true_user_min).min())[0]
print("La valeur la plus proche trouvée est Ls_min = "
      + str(Ls_true[Lsmin_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

Lsmax_ind = np.where(abs(Ls_true-Ls_true_user_max)==
                  abs(Ls_true-Ls_true_user_max).min())[0]
print("La valeur la plus proche trouvée est Ls_max = "
      + str(Ls_true[Lsmax_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

# Code ------------------------------------------------------------
var = data.variables[varname][:,:,:]
vartmp = var[Lsmin_ind[0]:Lsmax_ind[0],:,:]
dataplt = np.mean(vartmp,axis=0)

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
if (earthtopo): ax.coastlines(resolution="110m",linewidth=1)
gl = ax.gridlines(linestyle='--',color='black',
             draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
clevs = np.linspace(vmin,vmax,29)
plt.contourf(longitude, latitude, dataplt,
             clevs, transform=ccrs.PlateCarree(),cmap="jet")
plt.title(r"Moyenne dans un intervalle de temps de "+varname, size=14)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label(r'K',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
plt.show()


## Coupe de la valeur moyenne d'un champ dans un intervalle de $L_s$

In [ ]:
# Paramètres utilisateurs -----------------------------------------
year_user = 1 # année de simulation à regarder
Ls_user_min = 0. # longitude solaire choisie
Ls_user_max = 360. # longitude solaire choisie

varname = 'temp'
vmin = 200
vmax = 350

# Code ------------------------------------------------------------
Ls_true_user_min = year_user*360. + Ls_user_min
Ls_true_user_max = year_user*360. + Ls_user_max

Lsmin_ind = np.where(abs(Ls_true-Ls_true_user_min)==
                  abs(Ls_true-Ls_true_user_min).min())[0]
print("La valeur la plus proche trouvée est Ls_min = "
      + str(Ls_true[Lsmin_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

Lsmax_ind = np.where(abs(Ls_true-Ls_true_user_max)==
                  abs(Ls_true-Ls_true_user_max).min())[0]
print("La valeur la plus proche trouvée est Ls_max = "
      + str(Ls_true[Lsmax_ind]-year_user*360.)
      + " pour l'année " + str(year_user))

# Code ------------------------------------------------------------
var = data.variables[varname][:,:,:,:]
p = data.variables['p'][:,:,:,:]

fig, ax = plt.subplots(figsize=(14,6))
xplt=latitude
yplt = np.squeeze(np.mean(np.mean(p[Ls_ind,:,:,:],axis=3),axis=2))
dataplt = np.squeeze(np.mean(np.mean(var[Lsmin_ind[0]:Lsmax_ind[0],:,:,:],axis=0),axis=2))
im = ax.contourf(xplt,yplt,dataplt,60,cmap='jet',vmax=vmax, vmin=vmin)
ax.set_title(varname)
ax.set_yscale('log')
ax.set(xlabel='Latitude', ylabel='p')
ax.invert_yaxis()
fig.colorbar(im,ax=ax)
# -----------------------------------------------------------------
